In [1]:
import numpy as np
from scipy.stats import unitary_group

import matplotlib.pyplot as plt
np.set_printoptions(precision=3, suppress=True, linewidth=150) #print less digits of floats for readability
%matplotlib inline
%config InlineBackend.figure_format='retina' # makes the plots look nicer

In [2]:
from strawberryfields.decomposition_tests import reck_decompose,reck_recompose,clement_decompose,clement_recompose,rectangle_compact_decompose,rectangle_compact_recompose


In [3]:
m = 5
U = unitary_group.rvs(m)
phases = reck_decompose(U)
U_out = reck_recompose(phases)
np.allclose(U, U_out, atol=1e-14, rtol=1e-14)

True

In [4]:
phases

{'m': 5,
 'phi_ins': {0: -0.05280320563236862,
  1: -0.9397345005638794,
  2: 3.4324509311794715,
  3: -5.224533914925181},
 'deltas': {(0, 0): -0.4610125983411385,
  (1, 0): -0.26808542996802814,
  (0, 1): -0.7288279186195281,
  (2, 0): -0.28969537494497344,
  (1, 1): -0.8720959334219888,
  (0, 2): -0.3500854811048244,
  (3, 0): -0.2362235910868697,
  (2, 1): -0.1988032236735464,
  (1, 2): -0.7407372269353625,
  (0, 3): -0.2952258862928683},
 'sigmas': {(0, 0): -2.603862022214159,
  (1, 0): -3.8037527882291666,
  (0, 1): 1.0261772668256466,
  (2, 0): 2.3586614581742977,
  (1, 1): 3.063061732245969,
  (0, 2): 0.23086946885987736,
  (3, 0): 3.0305876068172104,
  (2, 1): 2.424815776967848,
  (1, 2): -0.23086946885987758,
  (0, 3): 1.2188084178810896},
 'zetas': {0: -2.3840744005066115,
  1: 0.9945394689854282,
  2: -1.6108191906703946,
  3: 1.4522062012649335,
  4: -0.4625121012104346}}

In [7]:
import numpy as np
from strawberryfields.program import Program
from strawberryfields.ops import Rgate,BSgate
from strawberryfields.engine import Engine



prog = Program(m)

with prog.context as q:
    
    for j in range(m-1):
        phi_j = phases['phi_ins'][j]
        Rgate(phi_j) | q[j+1]
        for k in range(j+1):
            n = j - k
            delta = phases['deltas'][n,k]
            sigma = phases['sigmas'][n,k]
            phi1 = sigma+delta
            phi2 = sigma-delta
            BSgate(np.pi/4,np.pi/2) | (q[n],q[n+1])
            Rgate(phi1) | q[n]
            Rgate(phi2) | q[n+1]
            BSgate(np.pi/4,np.pi/2) | (q[n],q[n+1])
            ####phase -pi/2???
    for j in range(m):
        zeta = phases['zetas'][j]
        Rgate(zeta) | q[j]

eng = Engine("fock", backend_options={"cutoff_dim": 10})
# result = eng.run(prog)
# state = result.state

In [8]:
prog.draw_circuit()

['/Users/yuanyao/Desktop/notebooks/xanadu-Yuan/demo/strawberryfields/circuit_tex/output_2021_May_14_06:20PM.tex',
 '\\documentclass{article}\n\\pagestyle{empty}\n\\usepackage{qcircuit}\n\\begin{document}\n\\Qcircuit {\n & \\qw  & \\multigate{1}{BS}  & \\gate{R}  & \\multigate{1}{BS}  & \\qw  & \\qw  & \\qw  & \\multigate{1}{BS}  & \\gate{R}  & \\multigate{1}{BS}  & \\qw  & \\qw  & \\qw  & \\qw  & \\qw  & \\qw  & \\multigate{1}{BS}  & \\gate{R}  & \\multigate{1}{BS}  & \\qw  & \\qw  & \\qw  & \\qw  & \\qw  & \\qw  & \\qw  & \\qw  & \\qw  & \\multigate{1}{BS}  & \\gate{R}  & \\multigate{1}{BS}  & \\gate{R}  & \\qw \\\\\n & \\gate{R}  & \\ghost{BS}  & \\gate{R}  & \\ghost{BS}  & \\multigate{1}{BS}  & \\gate{R}  & \\multigate{1}{BS}  & \\ghost{BS}  & \\gate{R}  & \\ghost{BS}  & \\qw  & \\qw  & \\qw  & \\multigate{1}{BS}  & \\gate{R}  & \\multigate{1}{BS}  & \\ghost{BS}  & \\gate{R}  & \\ghost{BS}  & \\qw  & \\qw  & \\qw  & \\qw  & \\qw  & \\qw  & \\multigate{1}{BS}  & \\gate{R}  & \\multig

In [22]:
import strawberryfields as sf

In [24]:
sf.ops.BSgate(np.pi/4,0).H

In [44]:
m = 12
U = unitary_group.rvs(m)
phases = clement_decompose(U)
U_out = clement_recompose(phases)
np.allclose(U, U_out, atol=1e-14, rtol=1e-14)

True

In [5]:
m = 5
U = unitary_group.rvs(m)
phases = rectangle_compact_decompose(U)
U_out = rectangle_compact_recompose(phases)
np.allclose(U, U_out, atol=1e-14, rtol=1e-14)

True

In [6]:
phases

{'m': 5,
 'phi_ins': {0: 2.960601871152557, 2: -1.6715107531897353},
 'deltas': {(0, 0): -1.1826787920579578,
  (2, 4): 1.3040724346248338,
  (3, 3): 0.5222974112768394,
  (2, 0): -1.232696887144203,
  (1, 1): -0.858541460362257,
  (0, 2): -0.41940959311630094,
  (0, 4): 0.8247359781891787,
  (1, 3): 0.20627067588372291,
  (2, 2): 0.44238981442414343,
  (3, 1): 0.5610217115633193},
 'sigmas': {(0, 0): -2.433435971070339,
  (2, 4): 1.3893940124275552,
  (3, 3): -0.026389067297590163,
  (2, 0): -4.3510226947703785,
  (1, 1): 1.1073822367222357,
  (0, 2): 4.291445379194071,
  (0, 4): 3.289326127084394,
  (1, 3): -0.18573990190202982,
  (2, 2): -1.1963175020224082,
  (3, 1): -2.2730116815454844},
 'phi_outs': {3: 1.4536689929039366, 1: 1.2612197324991068},
 'phi_edges': defaultdict(float,
             {(4, 4): 2.639376181136206,
              (4, 2): -0.1837411709004213,
              (4, 0): 0.5134764318801284})}

In [47]:
def M(n, sigma, delta, m):
    r"""
    The Bell M matrix from Eq 1 of the paper
    n here represents the starting mode of MZI
    """
    mat = np.identity(m, dtype=np.complex128)
    mat[n, n] = np.exp(1j * sigma) * np.sin(delta)
    mat[n, n+1] = np.exp(1j * sigma) * np.cos(delta)
    mat[n+1, n] = np.exp(1j * sigma) * np.cos(delta)
    mat[n+1, n+1] = -np.exp(1j * sigma) * np.sin(delta)
    return mat

def P(j, phi, m):
    mat = np.identity(m, dtype=np.complex128)
    mat[j,j] = np.exp(1j * phi)
    return mat
    
def is_unitary(U):
    return np.allclose(U @ U.conj().T, np.eye(U.shape[0]))

def check_square(U):
    return U.shape[0] == U.shape[1]

In [12]:
def clements_decompose(U):
    
    if not U.shape[0] == U.shape[1]:
        raise Exception('Matrix is not square')
            
    V = U.conj()
    
    m = U.shape[0]
        
    phases = dict()
    phases['m'] = m
    phases['phi_ins'] = dict() # mode : phi
    phases['deltas'] = dict() # (mode, layer) : delta
    phases['sigmas'] = dict() # (mode, layer) : sigma
    phases['zetas'] = dict() # mode : zeta
    phases['phi_outs'] = dict() # mode : phi
        
    for j in range(m-1):
        if j % 2 == 0:
            x = m - 1
            y = j
            phi_j = np.angle(V[x, y+1]) - np.angle(V[x, y]) # reversed order from paper
            V = V @ P(j, phi_j, m)
            phases['phi_ins'][j] = phi_j
            for k in range(j+1):
                if V[x,y] == 0:
                    delta = 0.5 * np.pi
                else:
                    delta = np.arctan2(-abs(V[x,y+1]), abs(V[x,y]))
                n = j - k
                V_temp = V @ M(n, 0, delta, m)
                sigma = np.angle(V_temp[x-1, y-1]) - np.angle(V_temp[x-1,y])
                V = V @ M(n, sigma, delta, m)
                phases['deltas'][n,k] = delta
                phases['sigmas'][n,k] = sigma
                x -= 1
                y -= 1
        else:
            x = m - j - 1
            y = 0
            phi_j = np.angle(V[x-1,y]) - np.angle(V[x,y])
            V = P(x, phi_j, m) @ V
            phases['phi_outs'][x] = phi_j
            for k in range(j+1):
                if V[x,y] == 0.:
                    delta = 0.5 * np.pi 
                else:
                    delta = np.arctan2(abs(V[x-1,y]), abs(V[x,y]))
                V_temp = M(x-1, 0, delta, m) @ V
                n = m + k - j - 2
                if j != k:
                    sigma = (np.angle(V_temp[x+1, y+1]) - np.angle(V_temp[x,y+1]))
                else:
                    sigma = 0
                phases['deltas'][n,m-k-1] = delta
                phases['sigmas'][n,m-k-1] = sigma
                V = M(n, sigma, delta, m) @ V
                x += 1
                y += 1

    # these next two lines are just to remove a global phase
    zeta = - np.angle(V[0,0])
    V = V @ P(0, zeta, m)
    phases['zetas'][0] = zeta

    for j in range(1,m):
        zeta = np.angle(V[0,0]) - np.angle(V[j,j])
        V = V @ P(j, zeta, m)
        phases['zetas'][j] = zeta
        
    if not np.allclose(V, np.eye(m)): #is_unitary(V):
        raise Exception('decomposition failed')
    return phases

In [13]:
def clements_recompose(phases):
    m = phases['m']
    U = np.eye(m, dtype=np.complex128)
    
    #upper left of interferometer
    for j in range(0,m-1,2):
        phi_j = phases['phi_ins'][j]
        U = P(j, phi_j, m) @ U
        for k in range(j+1):
            n = j - k
            delta = phases['deltas'][n,k]
            sigma = phases['sigmas'][n,k]
            U = M(n, sigma, delta, m) @ U
            
    #diagonal phases
    for j in range(m):
        zeta = phases['zetas'][j]
        U = P(j, zeta, m) @ U
    
    #lower right of interferometer
    for j in reversed(range(1,m-1,2)):
        for k in reversed(range(j+1)):
            n = m + k - j - 2
            delta = phases['deltas'][n,m-k-1]
            sigma = phases['sigmas'][n,m-k-1]
            U = M(n, sigma, delta, m) @ U
            
    for j in range(1,m-1,2):
        x = m - j - 1
        phi_j = phases['phi_outs'][x]
        U = P(x, phi_j, m) @ U
    return U

In [14]:
def absorb_zeta(phases):
    m = phases['m']
    new_phases = phases.copy()
    del new_phases['zetas']
    new_phases['phi_edges'] = defaultdict(float) # (mode, layer) : phi
    
    if m % 2 == 0:
        new_phases['phi_outs'][0] = phases['zetas'][0]
        for j in range(1,m):
            zeta = phases['zetas'][j]
            layer = m - j 
            for mode in range(j,m-1,2):
                new_phases['sigmas'][mode, layer] += zeta
            for mode in range(j+1,m-1,2):
                new_phases['sigmas'][mode, layer-1] -= zeta
            if layer % 2 == 1:
                new_phases['phi_edges'][m-1, layer] += zeta
            else:
                new_phases['phi_edges'][m-1, layer-1] -= zeta
    else:
        for j in range(m):
            zeta = phases['zetas'][j]
            layer =  m - j - 1
            for mode in range(j,m-1,2):
                new_phases['sigmas'][mode, layer] += zeta
            for mode in range(j+1,m-1,2):
                new_phases['sigmas'][mode, layer-1] -= zeta
            if layer % 2 == 0:
                new_phases['phi_edges'][m-1, layer] += zeta
            else:
                new_phases['phi_edges'][m-1, layer-1] -= zeta
    return new_phases        

In [15]:
def rectangle_compact_decompose(U):
    phases_temp = clements_decompose(U)
    return absorb_zeta(phases_temp)

def rectangle_compact_compose(phases):
    m = phases['m']
    U = np.eye(m, dtype=np.complex128)
    for j in range(0,m-1,2):
        phi = phases['phi_ins'][j]
        U = P(j, phi, m) @ U
    for layer in range(m):
        if (layer + m + 1) % 2 == 0:
            phi_bottom = phases['phi_edges'][m-1, layer]
            U = P(m-1, phi_bottom, m) @ U
        for mode in range(layer % 2, m-1, 2):
            delta = phases['deltas'][mode, layer]
            sigma = phases['sigmas'][mode, layer]
            U = M(mode, sigma, delta, m) @ U 
    for j, phi_j in phases['phi_outs'].items():
        U = P(j, phi_j, m) @ U
    return U

In [10]:
from thewalrus.random import random_banded_interferometer

In [51]:
def Reck_decomposition(U):
    r"""Decomposition of a unitary into the Reck scheme with sMZIs and phase-shifters.

    Args:
        U (array): unitary matrix

    Returns:
        dict[]: returns a dictionary contains all parameters
            where the keywords:
            * ``m``: the length of the matrix
            * ``phi_ins``:
            * ``deltas``:
            * ``sigmas``:
            * ``zetas``:
            * ``phi_outs``:

    """

    
    V = U.conj()
    m = U.shape[0]
    
    phases = dict()
    phases['m'] = m
    phases['phi_ins'] = dict() # mode : phi
    phases['deltas'] = dict() # (mode, layer) : delta
    phases['sigmas'] = dict() # (mode, layer) : sigma
    phases['zetas'] = dict() # mode : zeta

    for j in range(m-1):
        x = m - 1
        y = j
        phi_j = - np.angle(V[x, y+1]) + np.angle(V[x, y])
        Pj = P(j+1, phi_j, m)
        phases['phi_ins'][j] = phi_j
        V = V @ Pj
        for k in range(j+1):
            n = j - k
            delta = np.arctan(-V[x,y+1] / V[x,y]).real
            V_temp = V @ M(n, 0, delta, m)
            sigma = np.angle(V_temp[x-1, y-1]) - np.angle(V_temp[x-1,y])
            phases['deltas'][n,k] = delta
            phases['sigmas'][n,k] = sigma
            V = V @ M(n, sigma, delta, m)
            x -= 1
            y -= 1
        
    # these next two lines are just to remove a global phase
    zeta = - np.angle(V[0,0])
    phases['zetas'][0] = zeta
    V = V @ P(0, zeta, m)

    for j in range(1,m):
        zeta = np.angle(V[0,0]) - np.angle(V[j,j])
        phases['zetas'][j] = zeta
        V = V @ P(j, zeta, m)
        
    if not np.allclose(V, np.eye(m)): #is_unitary(V):
        raise Exception('decomposition failed')
    return phases
    
def Reck_reconstruction(phases):
    m = phases['m']
    U = np.identity(m, dtype=np.complex128)
    for j in range(m-1):
        phi_j = phases['phi_ins'][j]
        U = P(j+1, phi_j, m) @ U
        for k in range(j+1):
            n = j - k
            delta = phases['deltas'][n,k]
            sigma = phases['sigmas'][n,k]
            U = M(n, sigma, delta, m) @ U
    for j in range(m):
        zeta = phases['zetas'][j]
        U = P(j, zeta, m) @ U
    return U

In [52]:

m = 5
U = unitary_group.rvs(m)
print(U)
# U = np.eye(m)
# U = random_banded_interferometer(m, w=10)
phases = Reck_decomposition(U)
print(phases)
print(Reck_reconstruction(phases))


[[-0.336+0.17j   0.203-0.021j  0.254-0.01j  -0.077+0.698j  0.456+0.226j]
 [-0.492+0.284j -0.254+0.118j  0.183-0.397j  0.122-0.49j   0.127+0.37j ]
 [ 0.408+0.225j -0.354+0.406j -0.115+0.116j  0.302+0.283j -0.176+0.514j]
 [-0.165+0.251j  0.571+0.495j  0.174-0.097j  0.097+0.042j -0.508-0.17j ]
 [-0.409+0.246j  0.122-0.057j -0.491+0.661j  0.228-0.135j  0.059+0.047j]]
{'m': 5, 'phi_ins': {0: -3.034065780020868, 1: 0.005928249688905485, 2: -0.6181471678904045, 3: 3.654142599169832}, 'deltas': {(0, 0): -0.27529297589855056, (1, 0): -1.028962364473575, (0, 1): -0.49978239806426744, (2, 0): -0.2692333161335751, (1, 1): -0.12672125031634357, (0, 2): -0.6076601608773623, (3, 0): -0.07551163248935577, (2, 1): -0.5676399172132249, (1, 2): -0.7022312220826377, (0, 3): -0.6546555032870699}, 'sigmas': {(0, 0): 0.3955083031103004, (1, 0): 2.1212179322380256, (0, 1): -0.8799243698622932, (2, 0): 3.064680647727706, (1, 1): -3.6949393931784007, (0, 2): -0.984004309386475, (3, 0): -1.9458210550309396, (2, 

In [16]:
m = 30
U = unitary_group.rvs(m)
# U = np.eye(m)
# U = random_banded_interferometer(m, w=10)
phases = rectangle_compact_decompose(U)
U_out = rectangle_compact_compose(phases)
np.allclose(U, U_out, atol=1e-14, rtol=1e-14)

True

In [6]:
phases

{'m': 5,
 'phi_ins': {0: 4.6385534746361925,
  1: -3.693588845265942,
  2: 0.03304207013035332,
  3: 0.3545860097229374},
 'deltas': {(0, 0): -0.08009520984424659,
  (1, 0): -1.0470217533420352,
  (0, 1): -0.04291424939021244,
  (2, 0): -0.2946332070046099,
  (1, 1): -0.285966839030365,
  (0, 2): -0.6285106888995644,
  (3, 0): -0.18544715712929152,
  (2, 1): -0.4664875198494969,
  (1, 2): -1.021394600994785,
  (0, 3): -0.5313833804180761},
 'sigmas': {(0, 0): 0.3807872086511557,
  (1, 0): 2.483589183257826,
  (0, 1): 3.6435585657223353,
  (2, 0): 2.3759753601538103,
  (1, 1): -0.11630326314341466,
  (0, 2): 5.427296807806661,
  (3, 0): -2.755930004600666,
  (2, 1): 1.9000415052277404,
  (1, 2): 0.8558884993729254,
  (0, 3): -1.6262656189147018},
 'zetas': {0: 0.48198605427898544,
  1: -0.9959072714427082,
  2: -1.965868235683626,
  3: -2.075912524221969,
  4: -1.0157029019349657}}

In [67]:
phases

{'m': 3,
 'phi_ins': {0: 0.0},
 'deltas': {(0, 0): 1.5707963267948966,
  (0, 2): 1.5707963267948966,
  (1, 1): 1.5707963267948966},
 'sigmas': {(0, 0): 0.0, (0, 2): 0.0, (1, 1): 0.0},
 'phi_outs': {1: 0.0},
 'phi_edges': defaultdict(float, {(2, 2): 0.0, (2, 0): -3.141592653589793})}

In [107]:
import numpy as np
from strawberryfields.program import Program
from strawberryfields.ops import Rgate,sMZgate
from strawberryfields.engine import Engine


m=4

prog = Program(m)

with prog.context as q:
    
    for i,phi in enumerate(params_list['P_']):
        Rgate(phi) | q[i]

    for j in range(m-1):
        for k in range(len(params_list[j])):
            delta = params_list[j][k][0]
            sigma = params_list[j][k][1]
            theta1 = delta + sigma
            theta2 = sigma - delta
            sMZgate(theta1, theta2) | (q[j-k], q[j-k+1])
 
    for i,phi in enumerate(params_list['Q_']):
        Rgate(phi) | q[m-i-1]
        
eng = Engine("fock", backend_options={"cutoff_dim": 10})
result = eng.run(prog)
state = result.state

KeyError: 2

# Clements

In [9]:
from strawberryfields.decomposition_tests import Clement_decomposition,Clement_reconstruction

In [19]:
m=6
U = unitary_group.rvs(m)

In [11]:
U

array([[ 0.063-0.407j,  0.493+0.128j,  0.047+0.115j, -0.258-0.599j,
         0.039+0.346j, -0.096+0.018j],
       [ 0.335+0.229j,  0.055-0.254j, -0.269-0.35j ,  0.114-0.062j,
         0.373+0.445j,  0.468-0.029j],
       [-0.046-0.162j, -0.333-0.331j,  0.68 -0.102j,  0.333-0.229j,
         0.162+0.166j, -0.126-0.213j],
       [-0.47 +0.163j, -0.036-0.458j, -0.458+0.036j,  0.2  -0.396j,
        -0.313+0.025j, -0.109-0.151j],
       [-0.053-0.031j, -0.105-0.061j,  0.14 +0.19j ,  0.051-0.385j,
         0.026-0.425j,  0.605+0.478j],
       [-0.577+0.231j,  0.113+0.463j,  0.052+0.218j,  0.216+0.002j,
         0.434+0.142j,  0.183-0.218j]])

In [12]:
params_list = Clement_decomposition(U)

V matrix is unitary: [[ 1.-0.j  0.+0.j -0.-0.j -0.-0.j  0.-0.j  0.+0.j]
 [-0.-0.j  1.-0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j -0.-0.j  1.-0.j -0.+0.j -0.+0.j -0.+0.j]
 [-0.+0.j  0.-0.j -0.+0.j  1.-0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.-0.j  0.+0.j  1.-0.j  0.-0.j]
 [-0.-0.j -0.+0.j  0.+0.j  0.-0.j  0.-0.j  1.-0.j]]


In [13]:
def Clement_reconstruction(m,params_list):
    U_Q = np.identity(m, dtype=np.complex128)
    U_odd = np.identity(m, dtype=np.complex128)
    U_even = np.identity(m, dtype=np.complex128)

    for j in range(0,m-1,2):
        phi = params_list["P_"][j]
        U = P(j, phi, m) @ U
#         U_odd =  U_odd @ P(j,phi,m)
        for k in range(j+1):#range(len(params_list[j])):
            delta = params_list[j][k][0]
            sigma = params_list[j][k][1]
#             U_odd = U_odd@M(j-k, sigma, delta, m)
            U = M(j-k, sigma, delta, m) @ U
    for j in range(1,m-1,2):
        phi = params_list["P_"][j]
        U_even =    P(m-j-1, phi, m) @ U_even
        for k in range(len(params_list[j])):
            delta = params_list[j][k][0]
            sigma = params_list[j][k][1]
            U_even = M(m + k - j - 2, sigma, delta, m)@U_even
            

    for j in range(1,m):
        zeta = params_list["Q_"][j-1]
        U_Q = P(j, zeta, m) @ U_Q
        
    phase = P(0, params_list["global_zeta"][0], m)

    U_right = U_odd@phase@U_Q
    U_left = U_even
    return (U_right@U_left).T

array([[ 0.063-0.407j,  0.493+0.128j,  0.047+0.115j, -0.258-0.599j,
         0.039+0.346j, -0.096+0.018j],
       [ 0.335+0.229j,  0.055-0.254j, -0.269-0.35j ,  0.114-0.062j,
         0.373+0.445j,  0.468-0.029j],
       [-0.046-0.162j, -0.333-0.331j,  0.68 -0.102j,  0.333-0.229j,
         0.162+0.166j, -0.126-0.213j],
       [-0.47 +0.163j, -0.036-0.458j, -0.458+0.036j,  0.2  -0.396j,
        -0.313+0.025j, -0.109-0.151j],
       [-0.053-0.031j, -0.105-0.061j,  0.14 +0.19j ,  0.051-0.385j,
         0.026-0.425j,  0.605+0.478j],
       [-0.577+0.231j,  0.113+0.463j,  0.052+0.218j,  0.216+0.002j,
         0.434+0.142j,  0.183-0.218j]])

In [18]:
import numpy as np
from strawberryfields.program import Program
from strawberryfields.ops import Rgate,sMZgate, BSgate
from strawberryfields.engine import Engine


m=2

prog = Program(m)
eng = Engine('gaussian')

with prog.context as q:
    
    BSgate(0.25*np.pi,0) | (q[0], q[1])

result = eng.run(prog)

result.state.cov()

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [7]:
import sympy as s

In [8]:
theta1, theta2 = s.symbols(r'\theta_1 \theta_2', real=True)

In [9]:
BS = s.exp(-s.I * s.pi / 2) * s.Matrix([[1,s.I],[s.I,1]]) / s.sqrt(2)

In [20]:
BS

Matrix([
[-sqrt(2)*I/2,    sqrt(2)/2],
[   sqrt(2)/2, -sqrt(2)*I/2]])

In [10]:
middle = s.Matrix([[s.exp(s.I * (theta2+s.pi)), 0], [0, s.exp(s.I * (theta1))]])

In [21]:
middle

Matrix([
[exp(I*(\theta_2 + pi)),               0],
[                     0, exp(I*\theta_1)]])

In [11]:
sMZ = BS @ middle @ BS

In [12]:
sMZ

Matrix([
[     exp(I*\theta_1)/2 - exp(I*(\theta_2 + pi))/2, -I*exp(I*\theta_1)/2 - I*exp(I*(\theta_2 + pi))/2],
[-I*exp(I*\theta_1)/2 - I*exp(I*(\theta_2 + pi))/2,     -exp(I*\theta_1)/2 + exp(I*(\theta_2 + pi))/2]])

In [292]:
s.simplify((sMZ @ sMZ.conjugate().T)) # check unitary

Matrix([
[1, 0],
[0, 1]])

In [14]:
sigma, delta = s.symbols(r'\Sigma \delta', real=True)

In [15]:
theta1_new = sigma + delta
theta2_new = sigma - delta

In [16]:
sMZp = sMZ.subs(((theta1, theta1_new), (theta2, theta2_new)))

In [17]:
sMZp.simplify()
sMZp

Matrix([
[   exp(I*(\Sigma + \delta))/2 - exp(I*(\Sigma - \delta + pi))/2, -I*(exp(I*(\Sigma + \delta)) + exp(I*(\Sigma - \delta + pi)))/2],
[-I*(exp(I*(\Sigma + \delta)) + exp(I*(\Sigma - \delta + pi)))/2,   -exp(I*(\Sigma + \delta))/2 + exp(I*(\Sigma - \delta + pi))/2]])

In [19]:
s.simplify(sMZp)

Matrix([
[   exp(I*(\Sigma + \delta))/2 - exp(I*(\Sigma - \delta + pi))/2, -I*(exp(I*(\Sigma + \delta)) + exp(I*(\Sigma - \delta + pi)))/2],
[-I*(exp(I*(\Sigma + \delta)) + exp(I*(\Sigma - \delta + pi)))/2,   -exp(I*(\Sigma + \delta))/2 + exp(I*(\Sigma - \delta + pi))/2]])

In [18]:
s.simplify(sMZp / (s.exp(s.I * sigma)))

Matrix([
[cos(\delta),  sin(\delta)],
[sin(\delta), -cos(\delta)]])